# Capstone Project
### IBM Data Science Specialization - Course 9
_this notebook will be used for the Capstone Project - Battling Neighbourhoods_

## Importing libraries

In [206]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import geocoder

## Web scraping with BeautifulSoup library

### Part 1 - retrieving data from Wikipedia, preparing a DataFrame

In [207]:
#getting the sourse html of the webpage
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#creating the BS object to work with
soup = BeautifulSoup(source, 'lxml')

In [208]:
#fetching the table component from the html wikipedia page
table = soup.find("table") 

#each cell represents a row in the future DataFrame
cells = table.find_all("tr")

#we will make DataFrame from these arrays
PostalCode   = list()
Borough      = list()
Neighborhood = list()

#going through the cells and filling out the arrays
for i in range(1,len(cells)):
    current_elems = cells[i].find_all("td")
    
    nbhood = str(current_elems[2].text) #some issues with the last column 
    
    PostalCode.append(current_elems[0].text)
    Borough.append(current_elems[1].text)
    Neighborhood.append(nbhood[0:len(nbhood)-1])

In [209]:
data_ = {"PostalCode" : PostalCode, "Borough": Borough, "Neighborhood": Neighborhood}
df =pd.DataFrame(data=data_)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [210]:
#Processing the DataFrame according to given conditions

#1: all entries with Borough = Not Assigned must be dropped

df = df.loc[df['Borough'] != 'Not assigned'].reset_index()


#2: If neighborhood is unavailable, we copy borough into it

df.loc[df['Neighborhood'] == "Not assigned", 'Neighborhood'] = df['Borough']

df = df[['PostalCode','Borough','Neighborhood']]

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [211]:
#3: grouping neighborhoods by postal code

for elem in df['PostalCode'].tolist():
    #print('elem: ' + str(elem))
    borough = df.loc[df['PostalCode'] == elem,'Borough'].tolist()[0]
    neighborhoods = df.loc[df['PostalCode']==elem,'Neighborhood'].tolist()
    neighbString  = ""
    
    for i in neighborhoods:
        neighbString += (i+", ")
        
    df = df.loc[df['PostalCode'] != elem]
    df = df.append({'PostalCode': elem, 'Borough': borough, 'Neighborhood': neighbString[0:-2]},ignore_index=True)
    df.reset_index()

In [214]:
df.shape

(103, 3)

### Part 2 - adding latitude and longitude coordinates

Geocoder just kept returning Nones, so I opted for the CSV file way.

In [215]:
llframe = pd.read_csv("http://cocl.us/Geospatial_data")
llframe.head()  

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [216]:
postalcodes = df['PostalCode'].tolist()
latitudes   = list()
longitudes  = list()

for code in postalcodes:
        lat = llframe.loc[llframe['Postal Code'] == code, 'Latitude'].tolist()[0]
        lng = llframe.loc[llframe['Postal Code'] == code, 'Longitude'].tolist()[0]
        latitudes.append(lat)
        longitudes.append(lng)
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude']= pd.Series(longitudes,index=df.index)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Exploring neighborhoods 

We shall explore the neighborhoods in Toronto now. For that, we will first filter out all tuples that do not have word "Toronto" in their "Borough" attribute.

In [221]:
df = df.loc[df['Borough'].str.contains("Toronto")].reset_index()
df.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [249]:
df.shape

(38, 6)

Looks like it worked. Following in the footsteps of the course Lab, let's draw a map of Toronto

In [225]:
import folium
from geopy.geocoders import Nominatim

In [226]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [229]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=11)

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)
    
map_toronto

Very nice. Moving on to Foursquare.

In [230]:
CLIENT_ID = 'QF5S5RQX1YCVIDIRYRFA30W1HIMTKS40JN4TMDDG2WMO15WJ' 
CLIENT_SECRET = 'QYR5XESIJBB0DVXD3KTYRMNYQMM54RKEDZQ13Z0ZZDBNX5BH' 
VERSION = '20180605'
LIMIT = 100
#get infos about the venues near our neighborhood (within 500 m)
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [231]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [232]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Harbourfront, Regent Park",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Harbourfront, Regent Park",43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,"Harbourfront, Regent Park",43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,"Harbourfront, Regent Park",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
9,"Harbourfront, Regent Park",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


### Clustering of the neighborhoods

In [240]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="",prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()


(1697, 238)
(38, 238)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.0625,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0000,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00


In [241]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [252]:
kclusters=10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
len(kmeans.labels_)

38

In [253]:
map_clusters=folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'],df['Longitude'], df['Neighborhood'], kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters